<a href="https://colab.research.google.com/github/ynaowusu/protein-folding-quantum-algorithms/blob/main/proteinfolding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install qiskit #this is just me intalling qiskit into our notebook
!pip install matplotlib plotly #since it says we need a 3d structure to simulate the lattice and any other 3d elements
!pip install numpy


In [12]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random


In [13]:
"""
        proteins fold in 3D space and here we're working with a simplified lattice model w tetrahedral directions. There are two kinds of alternating sites in the protein chain: 'A' and 'B'.
        every turn corresponds to a direction in 3D space either x,y or z
        The vectors  [1, 1, 1]) are unit steps in different directions based on the protein structure model.
"""

class ProtienFoldVector:
  def __init_(self):
    self.lattice_vector = {
        "A":{
            0:[(1,1,1)], #|00> This is for the first round of qubits reprsents x,y,z
            1:[(1,-1,-1)], #|10>
            2:[(-1,1,-1)], #|01>
            3:[(-1,-1,-1)] #|11>
        },
        "B":{
            0:[(1,1,-1)], #|00>
            1:[(1,-1,1)], #|10>
            2:[(-1,1,1)], #|01>
            3:[(-1,-1,-1)] #11>
        }





    }

    pass


    #vectors go here


In [14]:
#hamiltonian functions


# t_i and t_j are vectors
# we decode the bit string and supply indices i and j directly.
#for each pair of beads i and j, T(i,j ) returns a 1 iff the turns t1 and tj are along the same axis
#we need this for the growth constrainst function
#we need this to prevrent the growth of the chain towards unphysical geometries



def T(t_i,t_j):
  if np.array_equal(t_i,t_j) or np.array_equal(t_i, -t_j) :
    return 1
  else:
    return 0

def test_T_function():
  assert T(np.array([1,1,1]),np.array([1,1,1]))== 1
  assert T(np.array([-1,-1,-1]),np.array([-1,-1,-1]))== 1
  assert T(np.array([1,1,1]),np.array([-1,-1,-1]))== 1

  assert T(np.array([1,-1,-1]),np.array([1,1,-1]))== 0
  assert T(np.array([-1,1,-1]),np.array([-1,-1,-1]))== 0

  print("all test_t_function tests passed")
test_T_function()

'''
because there are somethings we do not have implemented yet, growth_constraint_hamiltonian is a bit hardcoded
what growth constaint does is it eliminates sequences where the same axis
is chosen twice in a row, since this will give rise to a chain folding back onto itself.
turns is for now a placeholder but it  supposed to be a chain and so we will decode the measured qubits to then get the backbone turns
for the particular fold and then if some consecutive pairs share an axis, then the penalty which here is 30 will apply
N is the length of the peptide chain which is the number of amino-acid residues

'''
amino_acid_sequence = ([1,2,3,4,5,6])
N = random.choice(amino_acid_sequence)
def growth_constraint_Hamiltonian():
  H_gc = 0
  penalty_weight = 30
  turns = [np.array([1,1,1]),np.array([1,1,1]), np.array([1,-1,-1]),np.array([-1,-1,1]),np.array([1,1,-1]),np.array([1, -1,-1]),np.array([-1,-1,1])]
  for i in range(3, N - 1):
     same_axis = T(turns[i], turns[i + 1])
     H_gc += 30 * same_axis
  return H_gc
growth_constraint_Hamiltonian()

all test_t_function tests passed


0

In [15]:
#the methods we creae will be used to fold protein model with 6 and 8
#amino acid sequences on 3D lattice
amino_acid_sequence = ([6,7,8])
N = random.choice(amino_acid_sequence)
print(N)

8


In [19]:
from qiskit import QuantumCircuit

class CVARVQE:

    def __init__(self, hamiltonian, alpha = 0.1):
        # self is just the initializer
        # the hamiltonian calculates the amount of energy it takes to form any given protein configuration.
        # this is important because the optimizer needs a way to evaluate how "good" or "bad" a quantum circuit’s result is.
        # alpha = 0.1 means CVaR will only average the best 10% of outputs.
        self.hamiltonian = hamiltonian  # Save the energy calculator
        self.alpha = alpha              # Only use the best 10% of folds
        self.n_qubits = hamiltonian.total_qubits  # Know how many qubits to use in the circuit

    def create_ansatz(self, params):
        qc = QuantumCircuit(self.n_qubits)
        # This creates a quantum circuit with the number of qubits based on the protein

        # Step 1: Put each qubit into superposition using Hadamard gates
        for i in range(self.n_qubits):
            qc.h(i)  # Hadamard gate turns each qubit into a mix of 0 and 1

        # Step 2: Add a rotation gate to each qubit (this is how we "teach" the circuit how to fold the protein)
        param_1 = 0  # Keeps track of which parameter we're using from the list

        for i in range(self.n_qubits):
            qc.ry(params[param_1], i)  # apply RY (Y-axis) rotation to qubit i using params[param_1]
            param_1 += 1  # Move to the next angle in the list

        for i in range(self.n_qubits - 1): #what this is is that it's making a CNOT gate to make an entanglement between two qubits
          qc.cx(i, i+ 1 ) #this entanglement lets qubits share information which is important because in proteins, one fold affects nearby folds.

        for i in range(self.n_qubits):
          qc.ry(params[param_1], i) #this is a repeat of the other line of code that just rotates the qubit again after the cnot entangled gate is applied
          param_1 += 1


        return qc  # Return the final circuit
    def evaluate_energy(self, params, n_shots = 2000):
      #this function will be based around evalauting the energy of the circuit.
      #it takes in the list of angles from a cirq, nshot is for how many times the cirq should run and it returns an energy score.
      qc = self.create_ansatz(params)#




